In [1]:
import pandas as pd
from model_initialisation import initialize_model
from utils import CustomEmbeddings, CustomLLM
from langchain_community.vectorstores import FAISS


In [2]:
data = pd.read_csv('test_data/test_data.csv')
data.head()


,question
0,Who is the youngest driver to win a Formula 1 ...
1,How many world championships has Lewis Hamilto...
2,Which team did Max Verstappen make his F1 debu...
3,What is the nationality of Charles Leclerc? \n
4,Who replaced Daniel Ricciardo at McLaren in 20...


In [4]:
data.shape[0]

300

In [3]:
embeddings = CustomEmbeddings()
vectorstore = vectorstore = FAISS.load_local(
    'faiss_index', embeddings, allow_dangerous_deserialization=True
)
rag_chain = initialize_model(vectorstore)


In [6]:
rag_chain.invoke({'input': data['question'][0], 'chat_history': []})

{'input': 'Who is the youngest driver to win a Formula 1 race?  \n',
 'chat_history': [],
 'context': [Document(id='e2f02441-7a20-4e97-a707-663030dcdc3e', metadata={'title': 'Max Verstappen', 'summary': "Max Emilian Verstappen (Dutch pronunciation: [ˈmɑks fɛrˈstɑpə(n)]; born 30 September 1997) is a Dutch and Belgian racing driver, who competes under the Dutch flag in Formula One for Red Bull Racing. Verstappen has won four Formula One World Drivers' Championship titles, which he won consecutively from 2021 to 2024 with Red Bull, and has won 63 Grands Prix across 10 seasons.\nBorn in Hasselt and raised in Maaseik, Verstappen is the son of Dutch former Formula One driver Jos Verstappen and Belgian former kart racer Sophie Kumpen. After a successful karting career—culminating in his record-breaking 2013 season—Verstappen graduated to junior formulae. Progressing directly to FIA European Formula 3, Verstappen broke several records on his way to third in the championship in his rookie seaso

In [ ]:
data['answer'] = data['question'].apply(
    lambda x: rag_chain.invoke({'input': x, 'chat_history': []})['answer']
)

data.to_csv('data_with_answers.csv', index=False)


In [8]:
from settings import (
    API_HEADERS,
    GENERATOR_BASE_URL,
)
import requests
from langchain_core.language_models.llms import LLM


# не злись, так просто быстрее сейчас
class JudgeLLM(LLM):
    # TODO: добавить внутренние переменные GENERATOR_MODEL и тд.

    def _call(self, prompt: str, **kwargs) -> str:
        data = {
            'model': 'gpt-4o-mini',
            'messages': [
                {
                    'role': 'system',
                    'content': """You are an intelligent assistant tasked with evaluating whether a given answer appropriately addresses 
                                                the provided question. For each input, decide if the answer is correct, relevant, and sufficiently detailed based on 
                                                the question. If the answer is correct and sufficiently detailed, respond with "1" If the answer is 
                                                insufficient or incorrect, respond with "0" with an explanation of why it’s inadequate.
                                                """,
                },
                {'role': 'user', 'content': prompt},
            ],
            'temperature': 0.7,
        }
        try:
            response = requests.post(GENERATOR_BASE_URL, headers=API_HEADERS, json=data)
            response.raise_for_status()
            return response.json()['choices'][0]['message']['content']

        except requests.exceptions.RequestException as e:
            print(f'Error while generating answer: {e}')
            return 'Failed to generate an answer.'

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return 'custom'


In [9]:
main_judge = JudgeLLM()

In [13]:
data.loc[0]['question']

'Who is the youngest driver to win a Formula 1 race?  \n'

In [ ]:
main_judge.invoke(input=f'Question: {row['question']}; Answer: {row['answer']}'

In [15]:
data

,question,answer
0,Who is the youngest driver to win a Formula 1 ...,The youngest driver to win a Formula 1 race is...
1,How many world championships has Lewis Hamilto...,Lewis Hamilton has won a total of seven Formul...
2,Which team did Max Verstappen make his F1 debu...,Max Verstappen made his Formula 1 debut with t...
3,What is the nationality of Charles Leclerc? \n,Charles Leclerc is Monégasque. He was born and...
4,Who replaced Daniel Ricciardo at McLaren in 20...,Daniel Ricciardo was replaced at McLaren in 20...
...,...,...
295,How is F1 tackling the challenge of reducing t...,Formula 1 is addressing the challenge of reduc...
296,What advancements are expected in fan experien...,I don't have specific information on the advan...
297,What is the potential of driverless technologi...,The retrieved context does not provide specifi...
298,How are teams integrating VR and AR for race p...,Teams in Formula 1 are increasingly utilizing ...


In [ ]:
def judge_by_llm(row):
    ans = main_judge.invoke(
        input=f'Question: {row['question']}; Answer: {row['answer']}'
    )
    return ans


data['judge'] = data.apply(judge_by_llm, axis=1)

data.head()


,question,answer,judge
0,Who is the youngest driver to win a Formula 1 ...,The youngest driver to win a Formula 1 race is...,1
1,How many world championships has Lewis Hamilto...,Lewis Hamilton has won a total of seven Formul...,1
2,Which team did Max Verstappen make his F1 debu...,Max Verstappen made his Formula 1 debut with t...,1
3,What is the nationality of Charles Leclerc? \n,Charles Leclerc is Monégasque. He was born and...,1
4,Who replaced Daniel Ricciardo at McLaren in 20...,Daniel Ricciardo was replaced at McLaren in 20...,1


In [18]:
data.to_csv('main_metrics.csv', index=False)

In [20]:
data[data.judge != '1']

,question,answer,judge
10,Which team has the most constructors’ champion...,Ferrari holds the record for the most Construc...,0 - The answer is incorrect. As of October 202...
12,Who was Ferrari’s first world champion driver?...,Ferrari's first world champion driver was Gius...,0 - The answer is incorrect because it states ...
14,Which engine supplier does Aston Martin use in...,"In 2025, Aston Martin will continue to use Hon...",0 - The answer incorrectly states that Aston M...
27,How did Niki Lauda’s crash in 1976 change F1 s...,Niki Lauda's near-fatal crash at the Nürburgri...,0 - The answer is partially relevant but ultim...
29,Which track hosted the first night race in For...,The first night race in Formula 1 was held at ...,0 - The answer is incorrect. The first night r...
...,...,...,...
259,What is the record for the most lead changes i...,I don't know the answer to that question.,0 - The answer is inadequate because it does n...
294,What are the benefits of 3D printing for F1 pa...,I don't know.,0 - The answer is inadequate because it does n...
296,What advancements are expected in fan experien...,I don't have specific information on the advan...,0 - The answer does not directly address the q...
297,What is the potential of driverless technologi...,The retrieved context does not provide specifi...,0 - The answer does not specifically address t...


In [22]:
accuracy = (300 - 61) / 300
print(f'Accuracy: {accuracy:2f}')


Accuracy: 0.796667


In [23]:
mistakes = data[data.judge != '1']

In [27]:
for i in range(mistakes.shape[0]):
    row = mistakes.iloc[i]
    print(f'{i}-th Question: ', row['question'])
    print('Answer: ', row['answer'])
    print('Judge: ', row['judge'])
    print()


0-th Question:  Which team has the most constructors’ championships in F1 history?  

Answer:  Ferrari holds the record for the most Constructors' Championships in Formula One history, with a total of 16 titles. Their most recent championship win was in 2008.
Judge:  0 - The answer is incorrect. As of October 2023, the team with the most Constructors' Championships in F1 history is Mercedes, which has won 8 titles, while Ferrari has won 16 titles in total but only 8 of those were Constructors' Championships. Additionally, the answer lacks a current context regarding the latest championship standings, which is relevant for the question.

1-th Question:  Who was Ferrari’s first world champion driver?  

Answer:  Ferrari's first world champion driver was Giuseppe Farina, who won the inaugural Drivers' World Championship in 1950 while competing for Alfa Romeo. He narrowly defeated his teammate, Juan Manuel Fangio, in that championship.
Judge:  0 - The answer is incorrect because it states 